In [1]:
from pip._internal import main
main(['install','mysql-connector-python-rf'])
import mysql.connector
import requests
from bs4 import BeautifulSoup

Defaulting to user installation because normal site-packages is not writeable


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
base_url = 'https://247sports.com/'
url = base_url + "Season/2021-Football/CollegeTeamTalentComposite/?Conference=";
conferences = ['ACC','Big-12','Big-Ten','Pac-12','SEC','AAC']

In [5]:
def parse_team_info(soup):
    a = soup.find('a', {'class':'rankings-page__name-link'})
    teamName = a.string.strip()
    team_breakdown_url = a['href'] if a.has_attr('href') else None
    logo_container = soup.find('img')
    logo_url = logo_container['data-src'] if logo_container.has_attr('data-src') else None
    if logo_url is not None:
        logo_url = logo_url.split('?')[0]
    team_data = {'name':teamName,'team_url' : team_breakdown_url, 'logo_url' : logo_url}
    return team_data

def get_team_data():
    data = {}
    for conference in conferences:
        conference_url = url + conference
        html = requests.get(conference_url, headers={"User-Agent": "Requests"}).content
        soup = BeautifulSoup(html, 'html.parser')
        team_info_containers = soup.find_all('li', {'class' : 'rankings-page__list-item'})
        for team_container in team_info_containers:
            team_data = parse_team_info(team_container)
            team_data['conference'] = conference
            data[team_data['name']] = team_data
    return data

def get_team_talent_data(team_data, team_name):
    team_url = base_url + team_data['team_url']
    html = requests.get(team_url, headers={"User-Agent": "Requests"}).content
    soup = BeautifulSoup(html, 'html.parser')
    table_body = soup.find('tbody')
    # extract team members
    containers = table_body.find_all('td')
    roster = []
    for container in containers:
        a = container.find('a')
        name = a.string if a else container.find('span').string
        roster.append(name)
    
    players = []
    # extract talent info
    stats_container = soup.find('div', {'class' : 'scroll-table-container'})
    table_body = stats_container.find('tbody')
    rows = table_body.find_all('tr')
    if len(rows) != len(roster):
        print("##### error: mismach roster and stats")
    else:
        for i in range(len(rows)):
            name = roster[i]
            row = rows[i]
            stats = row.find_all('td')
            position = stats[1].string
            height = stats[2].string
            weight = stats[3].string
            year = stats[4].string
            stars = row.find_all('span', {'class':'icon-starsolid'})
            player = {'name':name, 'team': team_name, 'position' : position, 'height' : height, 'weight' : weight, 'year' : year, 'stars' : len(stars)}
            players.append(player)
    return players

In [7]:
# DATA
teams = get_team_data()
players = []

# get list of all players on all teams
for team_name in teams:
    roster = get_team_talent_data(teams[team_name], team_name)
    players.extend(roster)

In [53]:
connection = mysql.connector.connect(user='cfbuser', password='cfbuser',
                              host='127.0.0.1',
                              database='cfbdb')
cursor = connection.cursor()

In [9]:
# select 
cursor.execute("SELECT * FROM conference")
results = cursor.fetchall()
print(results)

[('AAC',), ('ACC',), ('Big-12',), ('Big-Ten',), ('Pac-12',), ('SEC',)]


In [ ]:
# Insert conferences into conference table
for name in conferences:
    sql = (
       "INSERT IGNORE INTO conference(name)"
       "VALUES (%s)"
    )
    data = (name,)
    cursor.execute(sql, data)
    connection.commit()

In [ ]:
# Insert team data into db

# create list of values for inserts
teams = []
for team_name in data:
    info = data[team_name]
    item = (info['name'], info['team_url'], info['logo_url'], info['conference'])
    teams.append(item)

# insert teams into db
sql = """INSERT INTO team (name, team_url, logo_url, conference_name) 
        VALUES (%s, %s, %s, %s) """

cursor.executemany(sql, teams)
connection.commit()
print(cursor.rowcount, "Record inserted successfully into team table")

In [18]:
cursor.execute('SELECT id, name FROM team')
records = cursor.fetchall()
teamIds = {}
for record in records:
    team_name = record[1]
    team_id = record[0]
    teamIds[team_name] = team_id

In [61]:
records = []
for p in players:
    teamId = teamIds[p['team']]
    height = None
    if p["height"]:
        hightArr = p['height'].split('-')
        height = int(hightArr[0]) * 12 + int(hightArr[1])
    record = (str(p['name']), str(p['position']), height, int(p['weight']), str(p['year']), str(p['stars']),teamId)
    records.append(record)

In [63]:
# insert players into db
sql = 'INSERT INTO player (name, position, height, weight, year, stars, team_id) VALUES (%s, %s, %s, %s,%s,%s,%s)'

cursor.executemany(sql, records)
connection.commit()
print(cursor.rowcount, " Records inserted successfully into player table")

8148  Records inserted successfully into player table


In [64]:
connection.close()